# Initial Conditions & Parameter Exploration for USR in Higgs Inflation

This notebook probes the Ultra-Slow-Roll (USR) dynamics in the Higgs Inflation model, specifically accounting for the dependence on potential parameters $\lambda$ and $\xi$.



## Objectives
1. **Parameter Dependence**: Explore how varying $\xi$ impacts the potential and dynamics.
2. **Initial Conditions Scan**: Systematically scan initial field values $\phi_i$ and velocities $y_i$ to identify USR trajectories.
3. **USR Characterization**: Visualize the evolution of drift parameter $\eta$ to confirm USR signatures ($\eta \approx -6$).

In [ ]:
import sys
import os
root_dir = os.path.abspath('..')
if root_dir not in sys.path:
    sys.path.insert(0, root_dir)

import numpy as np
import matplotlib.pyplot as plt
from inflation_models import HiggsModel
from inf_dyn_background import run_background_simulation, get_derived_quantities
from scipy.interpolate import interp1d
import matplotlib.colors as colors

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10, 6)

## 1. Effect of Coupling $\xi$ on Potential Scale

First, we check how changing the non-minimal coupling $\xi$ affects the potential scale and background evolution for fixed initial conditions.

In [ ]:
# Fixed Initial Conditions
phi_i = 5.5
y_i = -0.01

# Vary Xi
xi_values = [10, 100, 1000, 10000]
lam_val = 0.1

plt.figure(figsize=(10, 6))

for x_val in xi_values:
    # Instantiate model with specific lambda and xi
    # Note: The HiggsModel class now accepts these in __init__
    model = HiggsModel(lam=lam_val, xi=float(x_val))
    
    # Set initial field conditions (inherited from InflationModel)
    model.phi0 = phi_i
    model.yi = y_i
    
    # Run Simulation
    T_span = np.linspace(0, 60, 1000)
    sol = run_background_simulation(model, T_span)
    data = get_derived_quantities(sol, model)
    
    # Plot N vs Field
    # We plot phi(N) to see trajectory
    # Extract phi from solution (first column)
    phi_t = sol[:, 0]
    N_t = data['N']
    
    plt.plot(N_t, phi_t, label=f'$\xi={x_val}$')

plt.xlabel('e-folds N')
plt.ylabel('Field $\phi$')
plt.title(f'Field Evolution for varying $\xi$ ($\lambda={lam_val}, \phi_i={phi_i}, y_i={y_i}$)')
plt.legend()
plt.grid(True)
plt.show()

## 2. Phase Space Scan for USR

Now we perform a scan over initial velocities $|y_i|$ for a fixed large $\xi=1000$, looking for the Ultra-Slow-Roll phase defined by $\eta \approx -6$.

In [ ]:
# Model Parameters
model_xi = 1000.0
model_lam = 0.1

# Scan Arrays
phi_fixed = 5.8 # Starting somewhat high on the plateau
y_velocities = -np.logspace(-5, -1, 100) # Log spacing from 1e-5 to 0.1

min_etas = []
usr_durations = []

print(f"Scanning {len(y_velocities)} velocities for Xi={model_xi}...")

for y in y_velocities:
    model = HiggsModel(lam=model_lam, xi=model_xi)
    model.phi0 = phi_fixed
    model.yi = y
    
    # Run
    sol = run_background_simulation(model, np.linspace(0, 50, 1000))
    data = get_derived_quantities(sol, model)
    
    # Calculate eta
    eta = 2 * (data['epsH'] - data['etaH'])
    min_etas.append(np.min(eta))
    
    # Calculate USR duration (N where eta < -5)
    is_usr = eta < -5.0
    if np.any(is_usr):
        N = data['N']
        # Approximate integral diff
        dN = np.diff(N, prepend=N[0])
        duration = np.sum(dN[is_usr])
        usr_durations.append(duration)
    else:
        usr_durations.append(0.0)

print("Scan Complete.")

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('Initial Speed $|y_i|$ (log scale)')
ax1.set_ylabel('Minimum $\eta$', color=color)
ax1.semilogx(np.abs(y_velocities), min_etas, color=color, marker='.')
ax1.tick_params(axis='y', labelcolor=color)
ax1.axhline(-6, color='k', linestyle='--', label='USR Target (-6)', alpha=0.5)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('USR Duration (e-folds)', color=color)
ax2.semilogx(np.abs(y_velocities), usr_durations, color=color, marker='x', linestyle=':')
ax2.tick_params(axis='y', labelcolor=color)

plt.title(f'Transition to USR at $\phi_i={phi_fixed}, \xi={model_xi}$')
plt.grid(True, which="both", ls="-", alpha=0.3)
plt.show()

## 3. Detailed Trajectory of a Golden Point

Let's pick a velocity that yields a significant USR duration (from the plot above) and visualize the full phase space trajectory.

In [ ]:
# Select a candidate based on scan (e.g., peak duration or specific value)
# For now we pick one manually within the likely range
golden_idx = np.argmax(usr_durations)
golden_y = y_velocities[golden_idx]
print(f"Golden Velocity Candidate: {golden_y}")

model = HiggsModel(lam=model_lam, xi=model_xi)
model.phi0 = phi_fixed
model.yi = golden_y

sol = run_background_simulation(model, np.linspace(0, 50, 2000))
data = get_derived_quantities(sol, model)
N = data['N']
eta = 2 * (data['epsH'] - data['etaH'])
eps = data['epsH']

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(N, eta, label='$\eta$')
plt.plot(N, eps, label='$\epsilon$')
plt.axhline(-6, color='r', linestyle='--', alpha=0.5)
plt.ylim(-8, 2)
plt.xlabel('e-folds N')
plt.legend()
plt.title(f'Slow Roll Parameters ($|y_i| \approx {abs(golden_y):.2e}$)')

plt.subplot(1, 2, 2)
# Phase Space: phi vs dphi/dN
# y in code is dphi/dT. Convert to dphi/dN = y / H = y/z
phi = sol[:, 0]
y_t = sol[:, 1]
z_t = sol[:, 2]
dphi_dN = y_t / z_t

plt.plot(phi, dphi_dN)
plt.xlabel('Field $\phi$')
plt.ylabel('$d\phi/dN$')
plt.title('Phase Space Trajectory')
plt.grid()

plt.tight_layout()
plt.show()